<a href="https://colab.research.google.com/github/BanafshehHassani/Deep-learning-end-to-end-by-using-TensorFlow-Keras-Hyperopt-and-MLflow/blob/main/House_Price_Prediction_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Deep Learning: End-to-End House Price Prediction

Author: [Banafsheh Hassani](https://www.linkedin.com/in/banafsheh-hassani-7b063a129/)

This notebook demonstrates the process of building an end-to-end deep learning model for house price prediction using TensorFlow Keras, Hyperopt for hyperparameter tuning, and MLflow for experiment tracking and model management. The model is trained on the California Housing dataset from scikit-learn.

Table of Contents
Introduction
Setup and Installation
Data Loading and Preprocessing
Model Building
Model Training and Evaluation
Hyperparameter Tuning
Building the Final Model
Predicting House Prices
Model Deployment and Serving
Conclusion
1. Introduction
In this notebook, we aim to build a deep learning model that predicts house prices based on various features such as the number of rooms, location, and other attributes. The California Housing dataset will be used for training and evaluation.

2. Setup and Installation
First, we need to install the necessary dependencies. Run the following commands to install TensorFlow, MLflow, Hyperopt, and other required libraries:

In [2]:
!pip install --upgrade tensorflow
!pip install mlflow
!pip install hyperopt


Next, import the required libraries for the project:

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
import mlflow
import mlflow.keras
import mlflow.tensorflow
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from hyperopt import fmin, hp, tpe, STATUS_OK, SparkTrials


3. Data Loading and Preprocessing
Load the California Housing dataset from scikit-learn and split it into training and test sets. Perform feature scaling using StandardScaler:

In [4]:
cal_housing = fetch_california_housing()
X_train, X_test, y_train, y_test = train_test_split(cal_housing.data, cal_housing.target, test_size=0.2)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


4. Model Building
Define the architecture of the neural network model using TensorFlow Keras. In this example, we use a simple feedforward network:

In [5]:
def create_model():
    model = Sequential()
    model.add(Dense(20, input_dim=8, activation="relu"))
    model.add(Dense(20, activation="relu"))
    model.add(Dense(1, activation="linear"))
    return model

model = create_model()
model.compile(loss="mse", optimizer="Adam", metrics=["mse"])


5. Model Training and Evaluation
Train the initial model using the training data and evaluate it on the test data. Also, set up callbacks for model checkpoints, early stopping, and logging training metrics to TensorBoard:

In [ ]:
experiment_log_dir = "/dbfs/Banafshhassani@gmail.com/tb"
checkpoint_path = "/dbfs/Banafshhassani@gmail.com/keras_checkpoint_weights.ckpt"

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=experiment_log_dir)
model_checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor="loss", mode="min", patience=3)

history = model.fit(X_train, y_train, validation_split=.2, epochs=35, callbacks=[tensorboard_callback, model_checkpoint, early_stopping])

%load_ext tensorboard
%tensorboard --logdir $experiment_log_dir


Epoch 1/35
405/413 [============================>.] - ETA: 0s - loss: 1.4014 - mse: 1.4014
Epoch 1: val_loss improved from inf to 0.61794, saving model to /dbfs/Banafshhassani@gmail.com/keras_checkpoint_weights.ckpt
413/413 [==============================] - 8s 15ms/step - loss: 1.3854 - mse: 1.3854 - val_loss: 0.6179 - val_mse: 0.6179
Epoch 2/35
411/413 [============================>.] - ETA: 0s - loss: 0.5374 - mse: 0.5374
Epoch 2: val_loss improved from 0.61794 to 0.46937, saving model to /dbfs/Banafshhassani@gmail.com/keras_checkpoint_weights.ckpt
413/413 [==============================] - 7s 16ms/step - loss: 0.5378 - mse: 0.5378 - val_loss: 0.4694 - val_mse: 0.4694
Epoch 3/35
411/413 [============================>.] - ETA: 0s - loss: 0.4294 - mse: 0.4294
Epoch 3: val_loss improved from 0.46937 to 0.42418, saving model to /dbfs/Banafshhassani@gmail.com/keras_checkpoint_weights.ckpt
413/413 [==============================] - 4s 10ms/step - loss: 0.4290 - mse: 0.4290 - val_loss: 0.4

experiment_log_dir = "/dbfs/Banafshhassani@gmail.com/tb"
checkpoint_path = "/dbfs/Banafshhassani@gmail.com/keras_checkpoint_weights.ckpt"

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=experiment_log_dir)
model_checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor="loss", mode="min", patience=3)

history = model.fit(X_train, y_train, validation_split=.2, epochs=35, callbacks=[tensorboard_callback, model_checkpoint, early_stopping])

%load_ext tensorboard
%tensorboard --logdir $experiment_log_dir


6. Hyperparameter Tuning
Perform hyperparameter tuning using Hyperopt and MLflow. Define the search space and create an objective function for the tuning process:

In [ ]:
def create_model(n):
    model = Sequential()
    model.add(Dense(int(n["dense_l1"]), input_dim=8, activation="relu"))
    model.add(Dense(int(n["dense_l2"]), activation="relu"))
    model.add(Dense(1, activation="linear"))
    return model

def runNN(n):
    # ... (model training code)

space = {
    "dense_l1": hp.quniform("dense_l1", 10, 30, 1),
    "dense_l2": hp.quniform("dense_l2", 10, 30, 1),
    "learning_rate": hp.loguniform("learning_rate", -5, 0),
    "optimizer": hp.choice("optimizer", ["Adadelta", "Adam"])
}

spark_trials = SparkTrials()

with mlflow.start_run():
    best_hyperparam = fmin(fn=runNN, space=space, algo=tpe.suggest, max_evals=30, trials=spark_trials)


7. Building the Final Model
Build the final model using the best hyperparameters obtained from the tuning process. Modify the model architecture and compile it:

In [ ]:
import hyperopt

first_layer = hyperopt.space_eval(space, best_hyperparam)["dense_l1"]
second_layer = hyperopt.space_eval(space, best_hyperparam)["dense_l2"]
learning_rate = hyperopt.space_eval(space, best_hyperparam)["learning_rate"]
optimizer = hyperopt.space_eval(space, best_hyperparam)["optimizer"]

optimizer_call = getattr(tf.keras.optimizers, optimizer)
optimizer = optimizer_call(learning_rate=learning_rate)

def create_new_model():
    model = Sequential()
    model.add(Dense(first_layer, input_dim=8, activation="relu"))
    model.add(Dense(second_layer, activation="relu"))
    model.add(Dense(1, activation="linear"))
    return model

new_model = create_new_model()
new_model.compile(loss="mse", optimizer=optimizer, metrics=["mse"])


8. Predicting House Prices
Create a function that takes input features and returns the predicted house prices. Use this function to predict house prices for a sample input:

In [ ]:
def predict_house_prices(model, features):
    scaled_features = scaler.transform([features])
    predicted_price = model.predict(scaled_features)
    return predicted_price[0][0]

sample_input = [1.5, 3, 2, 1, 4, 5, 6, 7]  # Sample input features
predicted_price = predict_house_prices(new_model, sample_input)

print("Predicted House Price: $", predicted_price)


9. Model Deployment and Serving
To deploy and serve the model, you can use your preferred deployment method such as Flask or FastAPI to expose the model as a REST API endpoint.

10. Conclusion
In this notebook, we have demonstrated the end-to-end process of building a deep learning model for house price prediction using TensorFlow Keras, Hyperopt, and MLflow. The model was trained, evaluated, and tuned to obtain the best hyperparameters. Finally, we showcased how to use the trained model to predict house prices on new input data.